Teste para saber se usar um determinado seed faz todos os processadores gerarem os mesmos números aleatórios.

Em caso positivo, preciso repensar o algoritmo de CSA que coloquei no aguia4

In [1]:
using Distributed

nprocs = 3
addprocs(nprocs)
pids = workers()

3-element Array{Int64,1}:
 2
 3
 4

In [2]:
@everywhere using Random
@everywhere Random.seed!(1)

In [3]:
for w in pids
    @fetchfrom w println(rand(2))
end

      From worker 2:	[0.23603334566204692, 0.34651701419196046]
      From worker 3:	[0.23603334566204692, 0.34651701419196046]
      From worker 4:	[0.23603334566204692, 0.34651701419196046]


Conclusão: todo processador estará gerando o mesmo número aleatório, e isso pode prejudicar o algoritmo CSA porque na verdade não estão sendo feitos chutes distintos.

### Tentando colocar um seed para cada worker

In [4]:
for w in pids
    @spawnat w Random.seed!(w)
end

In [5]:
for w in pids
    @fetchfrom w println(rand(2))
end

      From worker 2:	[0.36679641243992434, 0.5238785747844117]
      From worker 3:	[0.8116984049958615, 0.9884323655013432]
      From worker 4:	[0.680079235935741, 0.8744368931121158]


Funcionou! Agora é só adaptar o código com os 20 processadores